# Perspective API

Mit der [Perspective API](https://www.perspectiveapi.com/) kann man den "Hasswert" einer Aussage mit Hilfe eines Machine-Learning Algorithmus messen. Dieser Algorithmus wurde von Jigsaw und Google kreiert und ist Teil einer grösseren Studie, welche Künstliche Intelligenz in Online-Diskussion zum Einsatz bringen möchte. Die Daten mit denen der Algorithmus "gefüttert" worden sind, basieren auf persönlichen Attacken, welche von der Plattfrom Wikipedia stammen und von "Menschen" verifiziert wurden. 

Zunächst müssen wir eine Library importieren, damit wir den Perspective API Server "requesten" d.h. abrufen könnnen. Dann definieren wir einen Satz und speichern diesen in einer Variablen __querytext__ (den Satz den wir überprüfen möchen).

In [4]:
import requests, json

querytext = "dialectics can break bricks"

Nun müssen wir die URL Adresse von dem Service definieren und einige andere spezifische Parameter deklarieren.

In [6]:
url = "https://commentanalyzer.googleapis.com/v1alpha1/comments:analyze?key=YOURAPIKEY"
data = {'comment': {'text': querytext}, 'requestedAttributes': {'TOXICITY':{}} }
headers = {'Content-type': 'application/json'}

Die **url** variable ist der link zur Perspective API welcher auch den API key beinhaltet "YOURAPIKEY". Es handelt sich um meinen eigenen API key (Please be careful ;)

Bisher haben wir nur Variablen definiert, nun machen wir endlich die Abfrage an den Server mit folgender Zeile:

In [7]:
r = requests.post(url, data=json.dumps(data), headers=headers)

In der Variable r ist nun die Antwort vom Server gespeichert. Versuchen wir mal die Variable auszugeben:

In [8]:
print (r)

<Response [200]>


Wir sollten **Response [200]** sehen, wenn alles gut ging. Response 200 heisst soviel, dass der Server erfolgreich geantwortet hat, im Prinzipt ist das nur eine Bestätigung dass die Seite gefunden und abgerufen wurde. Also, bisher alles gut. Habt ihr einen Fehler-Code 404, dann existiert die Seite nicht oder der API Key funktioniert nicht mehr oder ein anderer Fehler ist aufgetreten. Geben wir mal den Text der Antwort aus:

In [16]:
print (r.text)

{
  "attributeScores": {
    "TOXICITY": {
      "spanScores": [
        {
          "begin": 0,
          "end": 27,
          "score": {
            "value": 0.21176243,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.21176243,
        "type": "PROBABILITY"
      }
    }
  },
  "languages": [
    "en"
  ]
}



Hier sehen wir eine Datenstruktur mit einem Toxicity Score. Der Wert im value Feld zeigt die Toxizität zwischen 0.1 und 0.9 an. Die Schwierigkeit liegt nun darin, den Wert aus dieser Struktur herauszubekommen. Hierbei handelt es sich um eine JSON Datenstruktur. Mehr Infos gibts [hier](https://docs.python.org/2/library/json.html)

Speicheren wir zunächst die Antwort in einer Variablen. 

In [7]:
response = r.json()

Mit print überprüfen wir ob alles ok ging.

In [8]:
print (response)

{'attributeScores': {'TOXICITY': {'spanScores': [{'begin': 0, 'end': 27, 'score': {'value': 0.21176243, 'type': 'PROBABILITY'}}], 'summaryScore': {'value': 0.21176243, 'type': 'PROBABILITY'}}}, 'languages': ['en']}


Versucht nun mal mit der Syntax der eckigen Klammern die Syntax aufzubrechen. z. B.: 

In [9]:
print (response["attributeScores"]["TOXICITY"])

{'spanScores': [{'begin': 0, 'end': 27, 'score': {'value': 0.21176243, 'type': 'PROBABILITY'}}], 'summaryScore': {'value': 0.21176243, 'type': 'PROBABILITY'}}


Seht ihr den Unterschied? Es wird nun nur mehr noch ein Teil der JSON Struktur ausgegeben. Schlussendlich, um an den **Hasswert** heranzukommen, muss man folgendes schreiben:

In [3]:
print (response["attributeScores"]["TOXICITY"]["spanScores"][0]["score"]["value"])

0.7720496


übersichtlicher in einer Variablen, sieht es so aus:

In [10]:
hassFaktor = response["attributeScores"]["TOXICITY"]["spanScores"][0]["score"]["value"]
print(hassFaktor)

0.6901405


Der ganze Code nochmal zusammengefasst:

In [71]:
import requests, json

querytext = "dialectics can break bricks"

url = "https://commentanalyzer.googleapis.com/v1alpha1/comments:analyze?key=YOURAPIKEY"
data = {'comment': {'text': querytext}, 'requestedAttributes': {'TOXICITY':{}} }
headers = {'Content-type': 'application/json'}


r = requests.post(url, data=json.dumps(data), headers=headers)
response = r.json()

hassFaktor = response["attributeScores"]["TOXICITY"]["spanScores"][0]["score"]["value"]
print(hassFaktor)

0.035793785


Probiert mal die Variable __querytext__ zu verändern. Wie verändert sich der Wert? Versucht fünf Sätze-Paare zu finden, die ähnlich klingen bzw. ähnliche Strukturen haben, aber völlig entgegengesetzte Toxicity Werte ausgeben.

### Mehrere Sätze überprüfen mit Hilfe einer Liste

Man kann auch mehrere Sätze überprüfen, indem man sie in einer Liste (i. e. in einem Array) abspeichert. Mit einem for Loop kann jeder Eintrag/Satz aus dieser Liste nun überprüft werden:

In [10]:
import requests, json

beispielsaetze = ["Straight people shouldn't breed", "Gay people shouldn't breed."]

for satz in beispielsaetze:
    url = "https://commentanalyzer.googleapis.com/v1alpha1/comments:analyze?key=YOURAPIKEY"
    data = {'comment': {'text': satz}, 'requestedAttributes': {'TOXICITY':{}} }
    headers = {'Content-type': 'application/json'}

    r = requests.post(url, data=json.dumps(data), headers=headers)
    response = r.json()

    hassFaktor = response["attributeScores"]["TOXICITY"]["spanScores"][0]["score"]["value"]
    print(satz + ": " + str(hassFaktor))

Straight people shouldn't breed: 0.72420365
Gay people shouldn't breed.: 0.9027101


# Markov Ketten

[Markov Ketten](https://de.wikipedia.org/wiki/Markow-Kette) sind stochastische Modelle welche durch Kenntnis einer nur begrenzten Vorgeschichte Prognosen über die zukünftige Entwicklung möglich machen. Markov Ketten (Markov chains genannt im Englischen) werden in unterschiedlichen Gebieten angewendet, in der Wirtschaftsinformatik, Versicherungsmathematik um Vorhersagen zu machen, aber auch in der elektroakustischen Komposition (z. B. Xenakis) sowie beim Page Ranking (Google). Pyhton bietet eine einfache [Markov Ketten Generator Bibliothek](https://github.com/jsvine/markovify) an.

Die Bibliothek markovify kann vielseitig eingesetzt werden. Recht einfach und schnell können Twitter Bots programmiert werden (wie z. B. [@bern_trump_bot](https://twitter.com/bern_trump_bot), [@Mashbot](https://twitter.com/mashomatic), [@thought__leader](https://twitter.com/thought__leader)), aber auch als [Forum für Chatbots](https://www.reddit.com/r/SubredditSimulator/comments/3g9ioz/what_is_rsubredditsimulator/) oder für ein [fiktives Gefängnis-Manual](http://rebecca-ricks.com/2016/05/06/internal-security-zones/).

Markov-Ketten für die Generierung von Text sind aber relativ einfach und basieren auf dem Zufallsprinzip. Mit Machine Learning kann ein Computer mit gewissen Texten trainiert werden, welcher diese dann basierend auf dem "Gelernten" produzieren kann. Ein Einstiegspunkt ist dieser [Artikel](http://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/), der den Text von Alice im Wunderland verwendet, um den Computer zu trainieren. Ein Kunstprojekt von [Sascha Pohflepp](http://www.pohflepp.net) mit dem Titel [Recursion](http://www.pohflepp.net/Recursion) verwendet u. a. diese Technik.  

In [11]:
import markovify

text1 = """At the beginning of the second decade of the Twenty-First Century, global civilization faces 
a new breed of cataclysm. These coming apocalypses ridicule the norms and organisational structures of 
the politics which were forged in the birth of the nation-state, the rise of capitalism, and a 
Twentieth Century of unprecedented wars."""

text2 = """On our journey to connect the world, we often discuss products we're building and updates on our business. 
Today I want to focus on the most important question of all: are we building the world we all want? History is the story of how we've learned to come together in ever greater numbers -- from tribes to cities to 
nations. At each step, we built social infrastructure like communities, media and governments to empower us to 
achieve things we couldn't on our own. Today we are close to taking our next step. Our greatest opportunities are now global -- like spreading prosperity and freedom, promoting peace and understanding, lifting people out of poverty, and accelerating science. Our greatest challenges also need global responses -- like ending terrorism, fighting climate change, and preventing pandemics. Progress now requires humanity coming together not just as cities or nations, but also as a global community. 
"""

# Lade den Text in den Markow-Ketten Generator
text_model = markovify.Text(text1)

# Erstelle einen neuen Satz
print(text_model.make_sentence())

These coming apocalypses ridicule the norms and organisational structures of the Twenty-First Century, global civilization faces a new breed of cataclysm.


In [8]:
import markovify

txtfile = "./txt/zizek.txt"

# Textfile laden
with open(txtfile) as f:
    text = f.read()

# Lade den Text in den Markow-Ketten Generator
text_model = markovify.Text(text)

# Erzeuge fünf Zufallszahlen
#for i in range(5):
    #print(text_model.make_sentence())

# Erzeuge und gebe 10 Zufallssätze mit max. 140 Zeichen aus
for i in range(10):
    print(text_model.make_short_sentence(140))

So the paradox is, that it's not staged.
My big fear is that not only your elders, but also government and party officials!
Under Nazism, if you are buying a certain image of your social identity.
Don't wait for someone else to tell you what you desire -- it tells you how much.
The only true solution is therefore fully to accept the rules of prison life, however, is somewhat more refined.
If there is the Absolute?
The name of this trash can all the time.
This is why the problem with democratic rituals is homologous to the people, especially the young, in the pursuit of barbarity?
If there is a reality in today's Greece.
There is an old joke about socialism as the synthesis of the great problem of constitutional monarchy: how to desire.


Man kann aber auch zwei verschiedene Texte über Markow Ketten miteinander kombinieren. Hier ein Auszug aus der Dokumentation: 

With `markovify.combine(...)`, you can combine two or more Markov chains. The function accepts two arguments:

- `models`: A list of `markovify` objects to combine. Can be instances of `markovify.Chain` or `markovify.Text` (or their subclasses), but all must be of the same type.
- `weights`: Optional. A list — the exact length of `models` — of ints or floats indicating how much relative emphasis to place on each source. Default: `[ 1, 1, ... ]`.


In [1]:
import markovify

txtfile_a = "./txt/herzog.txt"
txtfile_b = "./txt/zizek.txt"

# Textfile laden
with open(txtfile_a) as f:
    text_a = f.read()
    model_a = markovify.Text(text_a)

with open(txtfile_b) as f:
    text_b = f.read()
    model_b = markovify.Text(text_b)

# 
model_combo = markovify.combine([ model_a, model_b ], [ 10, 1 ])
for i in range(10):
    print(model_combo.make_short_sentence(120))

I prefer to be Werner Herzog.
If you do not need to see them.
It's an ego trip, and I'm not out for the experience of adventure.
The Mexicans have a plane crash, it's not your own making.
It feels like just breathing or walking when I do other sorts of things.
I prefer to be Werner Herzog.
I would much rather eat potato chips on the guilt of white liberals.
And they have to be subversive or critical can be even worse.
And just releasing a film more accessible to mainstream audiences.
It feels like just breathing or walking when I do other sorts of things.


### Das ganze nun mit Twitter kombinieren

Nun kann uns erster kleiner Bot eine Markow Kette tweeten.

In [2]:
import tweepy, time
from config import config

# standard twitter api konfiguration
auth = tweepy.OAuthHandler(config['consumer_key'], config['consumer_secret'])
auth.set_access_token(config['access_token'], config['access_token_secret'])
api = tweepy.API(auth)

# markov chain generator
model_combo = markovify.combine([ model_a, model_b ], [ 1, 2 ])

# schleife
for i in range(10):
    tweetTXT = model_combo.make_short_sentence(120)
    print (tweetTXT)
    api.update_status(tweetTXT)
    # wir machen 5 sec. pause
    time.sleep(15)
    

It is far too simplistic to claim that the people all look, and then they politely ignore it.
Nonetheless, there is the proposed remedy tolerance, rather than as problems of inequality, exploitation, or injustice?
Under Nazism, if you are not buying a certain social status, ideology, and so on.
I know whenever it comes to be alive, so I'm cautious about taking risks.
What was his call to the company's success.
It's the same way as if you were a Jew, you are really with a storyboard.
But marriage was all a nightmare and so you'll never see me there.
You are guilty for who you are, you are not buying a certain image of your social identity.
I'm politically interested, but I do other sorts of things.
The material force of ideology makes me not see what I ironically call a slightly enlightened Buddhist hedonism.


~~Interessant wird es wenn wir Tweets direkt an Leute über deren Twitterhandle (mit dem @ Symbol) senden. Man kann mit der Tweepy API diese Handles z. B. direkt von einer Liste herausfinden. Dazu verwendet man die Funktion [list_members()](http://docs.tweepy.org/en/v3.5.0/api.html) von der Tweepy Bibliothek.~~ Aufgrund von diesem Skript bzw. diesem Verhalten wurde mein Account geblockt! Direct mentions an Leute die man nicht unbedingt followed werden von Twitter nicht gerne gesehen. Untenstehende Code also nicht ausführen.

In [12]:
import tweepy, time
from config import config

auth = tweepy.OAuthHandler(config['consumer_key'], config['consumer_secret'])
auth.set_access_token(config['access_token'], config['access_token_secret'])
api = tweepy.API(auth)

#members = api.list_members("@MashableNews ", "trump-s-america",-1)
#model_combo = markovify.combine([ model_a, model_b ], [ 1.5, 1 ])

#for i in range(10):
#    tweetTXT = "@"+ members[i].screen_name + " " + model_combo.make_short_sentence(120)
#    print (tweetTXT)
#    api.update_status(tweetTXT)
#    time.sleep(5)
    

ImportError: No module named 'tweepy'